# Q&A against Tabular Data from a CSV file  (experimental)

To really have a Smart Search Engine or Virtual assistant that can answer any question about your corporate documents, this "engine" must understand tabular data, aka, sources with tables, rows and columns with numbers. 
This is a different problem that simply looking for the top most similar results.  The concept of indexing, bringing top results, embedding, doing a cosine semantic search and summarize an answer, doesn't really apply to this problem.
We are dealing now with sources with Tables in which each row and column are related to each other, and in order to answer a question, all of the data is needed, not just top results.

In this notebook, the goal is to show how to deal with this kind of use cases. To continue with our Covid-19 theme, we will be using an open dataset called ["Covid Tracking Project"](https://covidtracking.com/data/download). The COVID Tracking Project dataset is a  CSV file that provides the latest numbers on tests, confirmed cases, hospitalizations, and patient outcomes from every US state and territory (they stopped tracking on March 7 2021).

Imagine that many documents on a data lake are tabular data, or that your use case is to ask questions in natural language to a LLM model and this model needs to get the context from a CSV file or even a SQL Database in order to answer the question. A GPT Smart Search Engine, must understand how to deal with this sources, understand the data and answer acoordingly.

In [38]:
import os
import pandas as pd
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import create_pandas_dataframe_agent
from langchain.agents import create_csv_agent

from common.prompts import CSV_PROMPT_PREFIX, CSV_PROMPT_SUFFIX

from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("credentials.env")

def printmd(string):
    display(Markdown(string))

In [39]:
# Set the Data source connection string. This is the location of the CSV with the COVID articles on each line. 
# You can change it and use your own data
DATASOURCE_CONNECTION_STRING = "DefaultEndpointsProtocol=https;AccountName=njonpdadls3;AccountKey=2MFKq8vzZVrxm4grSGo/cRbgkHqDuCFAoz0Yulx2D0wKBp879smmlrUZDcKeztL7M84pvQczHLWM+AStBLZ1SA==;EndpointSuffix=core.windows.net"
#DATASOURCE_SAS_TOKEN = "?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D"
DATASOURCE_SAS_TOKEN = "?sp=r&st=2023-08-29T23:09:13Z&se=2024-08-30T07:09:13Z&sv=2022-11-02&sr=c&sig=6peY%2BQUCV%2FfK5Rf6S86nt5chXpxnCJXkl2DVbaK0i2Q%3D"
#BLOB_CONTAINER_NAME = "arxivcs"
BLOB_CONTAINER_NAME = "customer360"
BLOB_FOLDER_NAME = "mapping"

In [5]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"]
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"]
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]
os.environ["OPENAI_API_TYPE"] = "azure"

## Download the dataset and load it into Pandas Dataframe

In [40]:
#Download the csv files to disk and inspect using pandas
import pandas as pd
remote_file_path = "https://njonpdadls3.blob.core.windows.net/customer360/mapping/teammapping.csv"

In [41]:
df = pd.read_csv(remote_file_path + DATASOURCE_SAS_TOKEN)
print("No. of lines:",df.shape[0])

No. of lines: 96


In [42]:
print("Rows and Columns:",df.shape)
df.head()

Rows and Columns: (96, 14)


,ATU,Sales Pod,Sales Territory,AE,Core SSP,Core CSA,AppInnov,App CSA,SSM core,DataAI SSP,SSM,DataAI CSA,acq CSA,DSR
0,Northwest,NWA Pod 2,US.NWA.100.DS.204,NESANC,WIARNSPI,AMRTAWFIK,JABERBER,ZACHARYZURLO,TBH,ERSCHULD,BKERR,EDOPOKU,YUNGC,RAPROZIN
1,Northwest,NWA Pod 2,US.NWA.100.DS.205,MAMARLOW,WIARNSPI,AMRTAWFIK,JABERBER,ZACHARYZURLO,TBH,ERSCHULD,BKERR,EDOPOKU,YUNGC,RAPROZIN
2,Retail,RTL Pod 1,US.RTL.W.200.DigitalSales.101,CHRHART,PRPETEN,AMRTAWFIK,CAGILL,ZACHARYZURLO,DANNYVO,GODAHL,DUCOTTON,FODERMID,BRVENAAS,MIA
3,Retail,RTL Pod 2,US.RTL.W.200.DigitalSales.201,ANGELMIRELES,PRPETEN,AMRTAWFIK,CAGILL,ZACHARYZURLO,DANNYVO,GODAHL,DUCOTTON,FODERMID,BRVENAAS,SEOGMAN
4,Retail,RTL Pod 2,US.RTL.W.200.DigitalSales.202,CHBERRY,PRPETEN,AMRTAWFIK,CAGILL,ZACHARYZURLO,DANNYVO,GODAHL,DUCOTTON,FODERMID,BRVENAAS,SEOGMAN


In [43]:
df.columns

Index(['ATU', 'Sales Pod', 'Sales Territory', 'AE', 'Core SSP', 'Core CSA',
       'AppInnov', 'App CSA', 'SSM core', 'DataAI SSP', 'SSM', 'DataAI CSA',
       'acq CSA', 'DSR'],
      dtype='object')

## Load our LLM and create our MRKL Agent

The implementation of Agents is inspired by two papers: the [MRKL Systems](https://arxiv.org/abs/2205.00445) paper (pronounced ‘miracle’ 😉) and the [ReAct](https://arxiv.org/abs/2210.03629) paper.

Agents are a way to leverage the ability of LLMs to understand and act on prompts. In essence, an Agent is an LLM that has been given a very clever initial prompt. The prompt tells the LLM to break down the process of answering a complex query into a sequence of steps that are resolved one at a time.

Agents become really cool when we combine them with ‘experts’, introduced in the MRKL paper. Simple example: an Agent might not have the inherent capability to reliably perform mathematical calculations by itself. However, we can introduce an expert - in this case a calculator, an expert at mathematical calculations. Now, when we need to perform a calculation, the Agent can call in the expert rather than trying to predict the result itself. This is actually the concept behind [ChatGPT Pluggins](https://openai.com/blog/chatgpt-plugins).

In our case, in order to solve the problem "How do I ask questions to a tabular CSV file", we need this REACT/MRKL approach, in which we need to instruct the LLM that it needs to use an 'expert/tool' in order to read/load/understand/interact with a CSV tabular file.

OpenAI opened the world to a whole new concept. Libraries are being created fast and furious. We will be using [LangChain](https://docs.langchain.com/docs/) as our library to solve this problem, however there are others that we recommend: [HayStack](https://haystack.deepset.ai/) and [Semantic Kernel](https://learn.microsoft.com/en-us/semantic-kernel/whatissk).

In [93]:
# Let's ask a complex question that requires multiple steps to solve it, and where it is not clear.
# looking at the dataframea above, even by a human on what columns to use
QUESTION = "How many AE for SDP ATU. Also list all the AE for SDP ATU"

In [94]:
# First we load our LLM: GPT-4 (you are welcome to try GPT-3.5-Turbo. You will see that GPT-3.5 
# does not have the cognitive capabilities to solve a complex question and will make mistakes)
#llm = AzureChatOpenAI(deployment_name="gpt-4", temperature=0, max_tokens=500)
llm = AzureChatOpenAI(deployment_name="gpt-35-turbo", temperature=0, max_tokens=500)

Now we need our agent and our expert/tool.  
LangChain has created an out-of-the-box agents that we can use to solve our Q&A to CSV tabular data file problem. For more informatio about tje **CSV Agent** click [HERE](https://python.langchain.com/en/latest/modules/agents/toolkits/examples/csv.html)

In [95]:
agent_executor = create_pandas_dataframe_agent(llm=llm,df=df,verbose=True)

In [96]:
agent_executor.agent.allowed_tools

['python_repl_ast']

In [97]:
printmd(agent_executor.agent.llm_chain.prompt.template)


You are working with a pandas dataframe in Python. The name of the dataframe is `df`.
You should use the tools below to answer the question posed of you:

python_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [python_repl_ast]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question


This is the result of `print(df.head())`:
{df_head}

Begin!
Question: {input}
{agent_scratchpad}

## Enjoy the response and the power of GPT-4 + REACT/MKRL approach

In [98]:
response = agent_executor.run(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX)



> Entering new  chain...
Thought: 
I need to set the pandas display options to show all the columns, then get the column names to see which columns I need to use to answer the question. Then I can filter the dataframe to only include rows where ATU is SDP, count the number of unique AE, and list all the unique AE.
Action: python_repl_ast
Action Input:
```
# set pandas display options
pd.set_option('display.max_columns', None)

# get column names
print(df.columns)
```
Observation: NameError: name 'pd' is not defined
Thought:I need to import pandas first before I can use it.
Action: python_repl_ast
Action Input:
```
import pandas as pd

# set pandas display options
pd.set_option('display.max_columns', None)

# get column names
print(df.columns)
```
Observation: Index(['ATU', 'Sales Pod', 'Sales Territory', 'AE', 'Core SSP', 'Core CSA',
       'AppInnov', 'App CSA', 'SSM core', 'DataAI SSP', 'SSM', 'DataAI CSA',
       'acq CSA', 'DSR'],
      dtype='object')

Thought:I need to filter t

In [99]:
# We are doing a for loop to retry N times. This is because: 
# 1) GPT-4 is still in preview and the API is being very throttled and 
# 2) Because the LLM not always gives the answer on the exact format the agent needs and hence cannot be parsed

for i in range(5):
    try:
        response = agent_executor.run(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX) 
        break
    except:
        response = "Error too many failed retries"
        print()
        continue
        
print(response)



> Entering new  chain...
Thought: 
I need to set the pandas display options to show all the columns, then get the column names to see which columns I need to use to answer the question. Then I can filter the dataframe to only include rows where ATU is SDP, count the number of unique AE, and list all the unique AE.
Action: python_repl_ast
Action Input:
```
# set pandas display options
pd.set_option('display.max_columns', None)

# get column names
print(df.columns)
```
Observation: Index(['ATU', 'Sales Pod', 'Sales Territory', 'AE', 'Core SSP', 'Core CSA',
       'AppInnov', 'App CSA', 'SSM core', 'DataAI SSP', 'SSM', 'DataAI CSA',
       'acq CSA', 'DSR'],
      dtype='object')

Thought:I need to filter the dataframe to only include rows where ATU is SDP, then count the number of unique AE and list all the unique AE.
Action: python_repl_ast
Action Input:
```
# filter dataframe to only include rows where ATU is SDP
sdp_df = df[df['ATU'] == 'SDP']

# count number of unique AE
num_ae = s

## Evaluation
Let's see if the answer is correct

In [13]:
#df['date'] = pd.to_datetime(df['date'])
july_2020 = df[(df['date'] >= '2020-07-01') & (df['date'] <= '2020-07-31')]
texas_hospitalized_july_2020 = july_2020[july_2020['state'] == 'TX']['hospitalizedIncrease'].sum()
nationwide_hospitalized_july_2020 = july_2020['hospitalizedIncrease'].sum()

In [14]:
print( "TX:",texas_hospitalized_july_2020,"Nationwide:",nationwide_hospitalized_july_2020)

TX: 0 Nationwide: 63105


It is Correct!

**Note**: Obviously, there were hospitalizations in Texas in July 2020 (Try asking ChatGPT), but this particular File, for some reason has 0 on the column "hospitalizedIncrease" for Texas in July 2020. This proves though that the model is NOT making up information or using prior knowledge, but instead using only the results of its calculation on this CSV file. That's what we need!

**Note 2**: You will also notice that if you run the above cell multiple times, not always you will get the same result. Sometimes it will even fail an error out. Why? 
1) This is still a very new field and LLMs and libraries still has a lot room to grow
2) Because for complex questions that require multiple steps to solve it, even humans make mistakes
3) Because if the column names are not clear, or ambiguous, or the data is not clean, it will make mistakes, just as humans would.

# Summary

So, we just solved our problem on how to ask questions in natural language to our Tabular data hosted on a CSV File.
With this approach you can see then that it is NOT necessary to make a dump of a database data into a CSV file and index that on a Search Engine, you don't even need to use the above approach and deal with a CSV data dump file. With the Agents framework, the best engineering decision is to interact directly with the data source API without the need to replicate the data in order to ask questions to it. Remember, GPT-4 can do SQL very well. 

Just think about this: if GPT-4 can do the above, imagine what GPT-5/6/7/8 will be able to do.

**Note**: We don't recommend using a pandas agent to answer questions from tabular data. It is not fast and it makes too many parsing mistakes. We recommend using SQL (see next notebook).

# Reference

- https://haystack.deepset.ai/blog/introducing-haystack-agents
- https://python.langchain.com/en/latest/modules/agents/agents.html
- https://tsmatz.wordpress.com/2023/03/07/react-with-openai-gpt-and-langchain/
- https://medium.com/@meghanheintz/intro-to-langchain-llm-templates-and-agents-8793f30f1837

# NEXT
We can see that GPT-4 is powerful and can translate a natural language question into the right steps in python in order to query a CSV data loaded into a pandas dataframe. 
That's pretty amazing. However the question remains: **Do I need then to dump all the data from my original sources (Databases, ERP Systems, CRM Systems) in order to be searchable by a Smart Search Engine?**

The next Notebook answers this question by implementing a Question->SQL process and get the information from data in a SQL Database, eliminating the need to dump it out.